In [20]:
!pip install openpyxl

In [21]:
import requests
import os
import datetime
import boto3
import pandas as pd
import numpy as np
import sys

In [22]:
DOWNLOAD_FILE = "dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx"
DOWNLOAD_DIR = "data/raw/gloss2pose/"
URL = "http://www.bu.edu/asllrp/" + DOWNLOAD_FILE
S3_BUCKET= "genasl-avatar"  #replacce with your bucket name
S3_PATH="v2/data/metadata/video_metadata.csv"

In [23]:
def download_file(download_dir, download_filename, url):
    os.makedirs(download_dir, exist_ok=True)

    download_path = os.path.join(download_dir, download_filename)

    response = requests.get(url)

    with open(download_path, "wb") as file_obj:
        file_obj.write(response.content)

    return download_path

In [24]:
def clean_asllvd_metadata(from_filepath, to_filepath):
    """
    Writes asllvd excel file to a cleaned csv
    """
    video_set = pd.read_excel(from_filepath)
    video_set = video_set.replace("============", np.nan)
    video_set = video_set.replace("------------", np.nan)
    video_set = video_set.replace("-------------------------", np.nan)
    video_set = video_set.dropna(axis=0, subset=["Gloss Variant", "Session", "Scene", "Start", "End"], how="all")
    new_video_set = video_set[["Main New Gloss.1","Gloss Variant", "Consultant", "Session", "Scene", "Start", "End"]]
    new_video_set = new_video_set.sort_values(by=["Main New Gloss.1","Gloss Variant", "Consultant", "Session", "Scene", "Start", "End"])
    # print(video_set)
    # print(new_video_set)
    new_video_set = new_video_set.reset_index().drop(["index"], axis=1)
    new_video_set["id"] = new_video_set.index
    new_video_set["Scene"] = new_video_set["Scene"].astype(int)
    new_video_set["Start"] = new_video_set["Start"].astype(int)
    new_video_set["End"] = new_video_set["End"].astype(int)
    new_video_set["session_scene"] = new_video_set['Session'].apply(str)+"-"+new_video_set['Scene'].apply(str)
    new_video_set["Scene"].apply(lambda x: str(x))
    new_video_set["session_scene_id"] = (
        new_video_set["session_scene"]
    ).astype("category").cat.codes
    new_video_set["is_corrupt"] = 0
    new_video_set["Main New Gloss"]= new_video_set["Main New Gloss.1"].astype(str)
    new_video_set.to_csv(to_filepath, index=False)

    return to_filepath

In [25]:
csv_filepath = os.path.join(DOWNLOAD_DIR, "video_metadata.csv")
filepath = download_file(DOWNLOAD_DIR, DOWNLOAD_FILE, URL)
csv_filepath = clean_asllvd_metadata(filepath, csv_filepath)
print("Uploading video_metadata to {}".format(S3_PATH))

s3 = boto3.resource("s3")
bucket = s3.Bucket(S3_BUCKET)

# Upload video metadata and timestamp
bucket.upload_file(csv_filepath, S3_PATH)
timestamp = s3.Object(S3_BUCKET, S3_PATH + ".timestamp")
timestamp.put(Body=r"{}".format(datetime.datetime.now()))


/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Uploading video_metadata to v2/data/metadata/video_metadata.csv


{'ResponseMetadata': {'RequestId': 'BPT5DFDV80TC75S8',
  'HostId': 'saGcQpwxRXpvcY1FWZGs+HoOykDKmHMWUG65Tu4ar5ZCKpIqDB0Y3OBSFu2CIw4Jn6wiTVVgJcE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'saGcQpwxRXpvcY1FWZGs+HoOykDKmHMWUG65Tu4ar5ZCKpIqDB0Y3OBSFu2CIw4Jn6wiTVVgJcE=',
   'x-amz-request-id': 'BPT5DFDV80TC75S8',
   'date': 'Mon, 08 Apr 2024 03:33:46 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"22da7d0d46745e50f32ce594e4793b5c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"22da7d0d46745e50f32ce594e4793b5c"',
 'ServerSideEncryption': 'AES256'}